<a href="https://colab.research.google.com/github/malinphy/os_llms_colab/blob/main/LaMini_Flan_T5_783M_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install --upgrade requests torch einops accelerate bitsandbytes
!pip install faiss-cpu -q
!pip install langchain -q
!pip install sentence-transformers -q

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ai4s_sm0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ai4s_sm0
  Resolved https://github.com/huggingface/transformers to commit 30ed3adf474aaf2972ab56f5624089bc24a6adf3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
os.listdir('drive/MyDrive/QQ_PROJECTS/turkish_pro_OS/data/data/vector_stores')

['sentence_transformers_clip_ViT_B_32_multilingual_v1',
 'cohere_multilingual',
 'faiss_index_OpenAI_pdfs_MC_500_30',
 'emrecan_bert_base_turkish_cased_mean_nli_stsb_tr',
 'muse_3',
 'faiss_index_MCdocs_en_500_30_translated_nllb-200-distilled-600M_embedded_all-MiniLM-L6-v2']

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
db = FAISS.load_local('drive/MyDrive/QQ_PROJECTS/turkish_pro_OS/data/data/vector_stores/faiss_index_MCdocs_en_500_30_translated_nllb-200-distilled-600M_embedded_all-MiniLM-L6-v2',
                      embeddings = embeddings)

In [8]:
query = '''What is the procedure for termination of the employer's contract?'''

In [9]:
x = db.similarity_search(query,k=4)
for i in x:
    print(i.page_content)

The employee may terminate the employment contract by giving a written explanation of the existence of another valid reason for termination and by complying with the notice period. In this case, the employee may bring proceedings under articles 17 to 21. The parties may always change the working conditions by agreement between them. The change in working conditions cannot be effectively enforced in the past.
for the employee who has been employed for more than three years, eight weeks after the notice is deemed to be terminated. These periods are minimized and may be increased by contracts. The party who does not comply with the notice requirement must pay compensation in the amount of the notice period. 8429 The employer may terminate the employment contract by paying the notice period's remuneration in advance.
the worker's wages shall not be paid within the period required to be expected for the termination of the employment contract. However, the provisions of the special laws in t

# LaMini-Flan-T5-783M

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch

checkpoint = "MBZUAI/LaMini-Flan-T5-783M"



tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,
                                             device_map='auto',
                                             torch_dtype=torch.float32)

pipe = pipeline('text2text-generation',
                 model = base_model,
                 tokenizer = tokenizer,
                 max_length = 1024,
                 do_sample=True,
                 temperature=0.3,
                 top_p=0.95,
                 )


In [11]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


# Try Your Prompts

In [12]:
# x = Rishi Sunak (born 12 May 1980) is a British politician serving as Prime Minister of the United Kingdom and
#                 Leader of the Conservative Party since 2022. He previously held two cabinet positions under Boris Johnson,
#                 lastly as Chancellor of the Exchequer from 2020 to 2022.
#                 Sunak has been the Member of Parliament (MP) for
#                 Richmond (Yorks) since 2015. He is the first British Asian and Hindu to hold the office of prime minister.
#                 Sunak was born in Southampton to parents of Indian descent who immigrated to Britain from East Africa in the 1960s.
#                 He was educated at Winchester College, studied philosophy, politics and economics at Lincoln College, Oxford,
#                 and earned an MBA from Stanford University in California as a Fulbright Scholar.
#                 During his time at Oxford University, Sunak undertook an internship at Conservative Central Office, and
#                 joined the Conservative Party. After graduating, Sunak worked for Goldman Sachs
#                 and later as a partner at the hedge fund firms The Children's Investment Fund Management and Theleme Partners.
#                 Sunak was elected to the House of Commons for Richmond in North Yorkshire at the 2015 general election. As a
#                 backbencher, Sunak supported the successful campaign for Brexit in the 2016 European Union (EU) membership referendum.
#                 Following the 2017 general election, Sunak was appointed to a junior ministerial position in Theresa May's second
#                 government as Parliamentary Under-Secretary of State for Local Government in the 2018 cabinet reshuffle.
#                 He voted three times in favour of May's Brexit withdrawal agreement, which was rejected by Parliament three times,
#                 leading to May announcing her resignation. During the 2019 Conservative Party leadership election,
#                 Sunak supported Johnson's successful bid to succeed May as Conservative leader and  prime minister, after which he
#                 appointed Sunak as Chief Secretary to the Treasury in July 2019.

In [17]:
%%time
import textwrap
response = ''
instruction = '''
            The employee may terminate the employment contract by giving a written explanation of the existence of another valid reason for
            termination and by complying with the notice period. In this case, the employee may bring proceedings under articles 17 to 21.
            The parties may always change the working conditions by agreement between them. The change in working conditions cannot be effectively
             enforced in the past.
            for the employee who has been employed for more than three years, eight weeks after the notice is deemed to be terminated.
            These periods are minimized and may be increased by contracts. The party who does not comply with the notice requirement must
            pay compensation in the amount of the notice period. 8429 The employer may terminate the employment contract by paying the
            notice period's remuneration in advance.
            the worker's wages shall not be paid within the period required to be expected for the termination of the employment contract.
            However, the provisions of the special laws in this regard shall be reserved. During this period, the termination of the employment
            contract shall be notified by the employer or the employee on the basis of another reason arising from the Law, even if the other
            party has been informed of the termination of the employment contract
            (a) the right of the worker to terminate the employment contract without notice to the employer in cases such as sickness,
            accident, childbirth and pregnancy, except for the reasons set out in subsection b) the stated conditions arise after six weeks
            after the notice periods in article 17 have been exceeded in accordance with the period of work of the worker at work.

            What is the procedure for termination of the employer's contract?
                '''
generated_text = pipe(instruction)
for text in generated_text:
  response += text['generated_text']
wrapped_text = textwrap.fill(response, width=1024)
print(wrapped_text)

The procedure for termination of the employer's contract is to pay the notice period's remuneration in advance, and the employee may bring proceedings under articles 17 to 21 if they do not comply with the notice requirement. However, the provisions of the special laws in this regard shall be reserved.
CPU times: user 23.6 s, sys: 84.8 ms, total: 23.7 s
Wall time: 35.3 s


In [14]:
%%%

[Document(page_content='The employee may terminate the employment contract by giving a written explanation of the existence of another valid reason for termination and by complying with the notice period. In this case, the employee may bring proceedings under articles 17 to 21. The parties may always change the working conditions by agreement between them. The change in working conditions cannot be effectively enforced in the past.', metadata={'source': 'c:\\Users\\orhan\\PycharmProjects\\langchain-tproject\\MCdocs\\kanunlar\\is_kanunu_calısma_2.pdf', 'file_path': 'c:\\Users\\orhan\\PycharmProjects\\langchain-tproject\\MCdocs\\kanunlar\\is_kanunu_calısma_2.pdf', 'page': 13, 'total_pages': 14, 'Author': 'Hp', 'Creator': 'Microsoft® Word 2010', 'CreationDate': "D:20230529140149+03'00'", 'ModDate': "D:20230529140149+03'00'", 'Producer': 'Microsoft® Word 2010'}),
 Document(page_content="for the employee who has been employed for more than three years, eight weeks after the notice is deemed

In [15]:
len('Rishi Sunak is a British politician who is currently serving as the Prime Minister of the United Kingdom. He was born on May 12, 1980, in Southampton to parents of Indian descent who immigrated to Britain from East Africa in the 1960s. Sunak was educated at Winchester College, studied philosophy, politics, and economics at Lincoln College, Oxford, and earned an MBA from Stanford University in California as a Fulbright Scholar. He was elected to the House of Commons for Richmond in North Yorkshire at the 2015 general election and has been the Member of Parliament (MP) for Richmond since 2015. Sunak supported')

614

In [16]:
%%time
import textwrap
response = ''
instruction = '''

                Rishi Sunak has ben the prme minister since 25 Octo.

                Correct the given sentence?
                '''
generated_text = pipe(instruction)
for text in generated_text:
  response += text['generated_text']
wrapped_text = textwrap.fill(response, width=240)
print(wrapped_text)

Rishi Sunak has been the prime minister since October 25th.
CPU times: user 3.69 s, sys: 10.5 ms, total: 3.7 s
Wall time: 3.74 s
